In [ ]:
list.files("/data/hts2018_pilot/Granek_4837_180427A5/")

In [ ]:
library(ShortRead)
library(tidyverse)
library(Biostrings)
set.seed(100)

In [ ]:
### Specify FASTQ file
myfqdir <- "/data/hts2018_pilot/Granek_4837_180427A5/"
myfqname <- "35_MA_P_S39_L002_R1_001.fastq.gz"
myfqfile <- file.path(myfqdir, myfqname)
outdir = path.expand("~/work/scratch/shortread")
dir.create(outdir)
tools::md5sum(myfqfile)

In [ ]:
### Read entire FASTQ file into memory
myfq <- readFastq(myfqfile)

### Check class
class(myfq)

In [ ]:
### Get Summary for FASTQ file
myfq

In [ ]:
### Get number of reads
length(myfq)

In [ ]:
### Subset based on first three records
myfq[1:3]

In [ ]:
### Subset based on three randomly selected records
myfq[sample(seq_len(length(myfq)),3, replace=FALSE)]

In [ ]:
### Get sequences and phred scores for reads 1,2, 3 and 1 (add duplicate on
### purpose
myrecs <- myfq[c(1,2,3,1)]

sread(myrecs) -> myseqs

quality(myrecs) -> myphreds

### Compare classes
class(myfq)
class(myseqs)
class(myphreds)

In [ ]:
### Check encoding of phred scores
myphreds %>% encoding

In [ ]:
myphreds[1]

In [ ]:
ShortRead::id(myrecs)

In [ ]:
as.character(myseqs)

In [ ]:
### Get the sequences and phred scores as character strings
myphreds

In [ ]:
### Check for duplicate reads
myseqs %>% duplicated

In [ ]:
### Get Unique reads
myseqs %>% unique

In [ ]:
### Reverse reads
myseqs %>% reverse

In [ ]:
### complement reads
myseqs %>% complement

In [ ]:
### Reverse complement reads
myseqs %>% reverseComplement

In [ ]:
### letter frequency counting
alphabetFrequency(myseqs)
letterFrequency(myseqs, c("A","T"))

In [ ]:
# alphabetByCycle(myrecs)
alphabetScore(myrecs)

# QC Report

In [ ]:
?list.files

In [ ]:
list.files(myfqdir,pattern = "1[0-1].*RZ.*.fastq.gz",full.names = TRUE) %>%
    qa(BPPARAM = MulticoreParam(workers=1)) %>%
    report(dest=file.path(outdir, "rz_10_11"))

In [ ]:
myfqfile %>%
    qa(BPPARAM = MulticoreParam(workers=1)) %>%
    report(dest=file.path(outdir, "35_MA_L002"))

In [ ]:
?qa

In [ ]:
?report

In [ ]:
BiocParallel::registered()

In [ ]:
?BiocParallelParam

In [ ]:
# myfqfile.qa = qa(myfqdir,pattern = "1[0-1].*RZ.*.fastq.gz",BPPARAM = MulticoreParam(workers=4))

In [ ]:
report(myfqfile.qa, dest=outdir)

In [ ]:
?qa

In [ ]:
### Some pattern matching
mypattern <- "AGTTGG"
matchPattern(mypattern, myseqs, max.mismatch=0)
matchPattern(mypattern, myseqs, max.mismatch=1)

In [ ]:
### Count occurences
countPattern(mypattern, myseq, max.mismatch=0) 
countPattern(mypattern, myseq, max.mismatch=1)

In [ ]:
### Alignment
pairwiseAlignment(mypattern, myseq)

In [ ]:
### How to stream FASTQ (rather than loading into memory
mystream <- FastqStreamer(myfqfile, 5)
mystream

### The first yield will retrieve records 1 through 5
yield(mystream)
### The second yield will retrieve records 6 through 10
yield(mystream)

### Be sure to close the stream
close(mystream)



### random Sampling (without reading in entire file upfront)

mysampler <- FastqSampler(myfqfile, 3)

### Each yield call will sample three records at random
yield(mysampler)
yield(mysampler)

close(mysampler)



sessionInfo()
q(save = "no")

# Streaming

In [ ]:
strm <- FastqStreamer(myfqfile)
repeat {
    fq <- yield(strm)
    if (length(fq) == 0)
    break
    ## Do something 
}

# Sampling

In [ ]:
sampler <- FastqSampler(myfqfile)
fq <- yield(sampler)
fq

In [ ]:
ShortRead::id(fq)